In [1]:
from collections import defaultdict
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from groq import Groq  # Make sure groq SDK is installed: pip install groq
from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env
api_key = os.getenv("GROQ_API_KEY")

# =========================
# 🔧 Setup: Embeddings & Vector DB
# =========================

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.load_local("panel_db", embeddings=embedding_model, allow_dangerous_deserialization=True)

C:\Users\adarw\AppData\Local\Temp\ipykernel_3912\2613876835.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\adarw\OneDrive\Documents\fypagile\panelAssignment\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# =========================
# 🔧 Setup: Groq API
# =========================

client = Groq(api_key=api_key)  # Replace with your key

def query_groq_for_panel_selection(panel_matches: dict, student_title: str, student_area: str) -> str:
    # Format context for LLM
    context = f"Student Project Title: {student_title}\nStudent Project Area: {student_area}\n\n"
    context += "Relevant Panel Data:\n"

    for panel, items in panel_matches.items():
        context += f"\nPanel {panel}:\n"
        for item in items:
            context += f"- {item}\n"

    # Prompt
    prompt = f"""
You are an academic panel recommender system. Based on the student’s title and project area, and the matching history (publication, grants, assignments) of each panel, recommend the top 5 most suitable panels and give a short reason for each.

{context}

Return the result in this format:

1. Panel X - Reason
2. Panel Y - Reason
...
"""

    # Query Groq LLM
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": "You are an intelligent assistant that recommends academic panels based on expertise."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4
    )

    return response.choices[0].message.content

In [3]:
# =========================
# 🔁 Main Flow Function
# =========================

def match_student_to_panel(student_title: str, student_area: str) -> str:
    query_text = f"{student_title} - {student_area}"
    query_embedding = embedding_model.embed_query(query_text)

    # Search top 10 most similar docs
    retrieved_docs = db.similarity_search_by_vector(query_embedding, k=10)

    # Group matched documents by panel
    panel_matches = defaultdict(list)
    for doc in retrieved_docs:
        panel = doc.metadata["lecturer_name"]
        source = doc.metadata["source"]
        panel_matches[panel].append(f"[{source.upper()}] {doc.page_content}")

    # Get recommendations from Groq
    result = query_groq_for_panel_selection(panel_matches, student_title, student_area)
    return result

In [4]:
student_title = "Enhancing Medical Imaging using GANs"
student_area = "Deep Learning in Healthcare"

recommendation = match_student_to_panel(student_title, student_area)
print(recommendation)

Based on the student's project title and area, and the matching history of each panel, I recommend the top 5 most suitable panels as follows:

1. Panel NOOR HIDAYAH BINTI ZAKARIA - Has experience in both deep learning in healthcare and GANs, with a specific focus on medical image analysis, making her a strong match for this project.

2. Panel FARHAN BIN MOHAMED - Has published multiple papers on GANs in medical imaging, including diabetic retinopathy lesions synthesis, which is closely related to the student's project.

3. Panel MOHD ADHAM BIN ISA - Has published a paper on a GAN-based data augmentation method for imbalanced multi-class skin lesion classification, which is relevant to the student's project on enhancing medical imaging using GANs.

4. Panel BANDER ALI SALEH AL-RIMY - Has published a paper on hierarchical knee image synthesis using GANs, which demonstrates expertise in medical image synthesis using GANs, a key aspect of the student's project.

5. Panel HAZA NUZLY BIN ABD

In [8]:
student_title = "AI-Powered Career Recommendation Web Platform for Fresh Graduates"
student_area = "Artificial Intelligence, Web Application Development"
 
recommendation = match_student_to_panel(student_title, student_area)
print(recommendation)


Based on the student's project title and area, and the matching history of each panel, I recommend the top 5 most suitable panels as follows:

1. Panel SHAHIDA SULAIMAN - Has experience in a similar project, TalentIQ - A Smart Recruitment System, which involves AI and web-based systems, making her a suitable panel for the student's AI-powered career recommendation web platform.

2. Panel FARKHANA BINTI MUCHTAR - Has experience in a project that involves AI and machine learning development for job seeker-employer connectivity, which is closely related to the student's project, making her a suitable panel.

3. Panel JOHAN BIN MOHAMAD SHARIF - Has experience in a project that involves AI-powered recommender systems and web application development, which aligns with the student's project, making him a suitable panel.

4. Panel SHAHLIZA BT. ABD. HALIM - Has experience in a similar project, TalentIQ - A Smart Recruitment System, which involves AI and web-based systems, making her a suitable 

In [9]:
student_title = "Brain Tumor Classification Using CNN and MRI Image Analysis"
student_area = "Deep Learning, Medical Imaging, Computer Vision"
 
recommendation = match_student_to_panel(student_title, student_area)
print(recommendation)

Based on the student's project title and area, and the matching history of each panel, here are the top 5 most suitable panels:

1. Panel NOR AZMAN BIN ISMAIL - Has two publications directly related to brain tumor classification using deep learning, making him an expert in the field.
2. Panel HUSSEIN SALEM ALI BIN SAMMA - Has a publication on a hybrid deep learning model for brain tumor classification, demonstrating his knowledge in the specific area of the student's project.
3. Panel ARAFAT MOHAMMED RASHAD - Co-authored a publication on a novel approach for classifying brain tumors combining a SqueezeNet model with SVM and fine-tuning, which is closely related to the student's project.
4. Panel HASLINA BINTI HASHIM - Has a publication on automated brain tumor detection using machine learning and has supervised a final year project on brain stroke classification using deep convolutional neural networks (CNN), showing her expertise in medical imaging and deep learning.
5. Panel IZYAN IZ

In [10]:
student_title = "Secure File Sharing System Using Advanced Encryption Standard (AES)"
student_area = "Cybersecurity, Data Encryption, Network Security"
 
recommendation = match_student_to_panel(student_title, student_area)
print(recommendation)


Based on the student's project title and area, as well as the matching history of each panel, I recommend the top 5 most suitable panels as follows:

1. Panel AHMAD FARIZ BIN ALI - This panel has direct experience with a project involving AES algorithm, which is a key component of the student's project, making them a strong fit.

2. Panel MUHALIM BIN MOHAMED AMIN - Similar to Panel AHMAD FARIZ, this panel has experience with a project that involves AES algorithm, increasing the likelihood of a good match.

3. Panel FUAD ABDULGALEEL ABDOH GHALEB - This panel has experience with a project that specifically uses AES method for end-to-end encryption, which aligns closely with the student's project goals.

4. Panel NORAFIDA BT. ITHNIN - With experience in a project that involves AES method for end-to-end encryption, this panel is well-suited to provide guidance on the student's project.

5. Panel MOHD. MURTADHA BIN MOHAMAD - This panel has experience with multiple projects related to file s

In [11]:
student_title = "Real-Time 3D Character Animation Using OpenGL and Skeletal Rigging"
student_area = "Computer Graphics, 3D Animation, Real-Time Rendering"
 
recommendation = match_student_to_panel(student_title, student_area)
print(recommendation)


Based on the student's project title and area, and the matching history of each panel, here are the top 5 most suitable panels:

1. Panel JUMAIL BIN TALIBA - The panel has a grant related to real-time character motion control, which aligns closely with the student's project on real-time 3D character animation using skeletal rigging.

2. Panel AHMAD FARIZ BIN ALI - The panel has a publication related to 3D object manipulation using speech and hand gesture, which shares similarities with the student's project in terms of 3D animation and real-time rendering.

3. Panel MD. SAH BIN HJ. SALAM - The panel has a publication related to 3D object manipulation using speech and hand gesture, which shares similarities with the student's project in terms of 3D animation and real-time rendering.

4. Panel SARINA BINTI SULAIMAN - The panel has experience with a FYP panel assignment on custom 3D character modeling application in augmented reality, which is closely related to the student's project on 3